In [1]:
import fenics as fen
import dolfin
import numpy as np
from examples.rectangular_waveguide.rectangular_waveguide import RectangularWaveguide

In [2]:
Lx, Ly = 5.0, 1.0
Nx, Ny = 201, 41
m = 2
g_N = fen.Expression('sin(x[1]*m*pi/Ly)', degree=2, Ly=Ly, m=m)
WG = RectangularWaveguide(Lx=Lx, Ly=Ly, Nx=Nx, Ny=Ny, g_N=g_N)
WG.setup()

In [3]:
N = 6
A = []
omegas = np.linspace(3, 4, N)
for omega in omegas:
    WG.solve(omega)
    A_sol = WG.get_solution()
    A.append(A_sol)

In [4]:
e0 = fen.Expression('1', degree=2)
e1 = fen.Expression('x[0]', degree=2)
E = []
for m in range(N):
    ei = fen.Function(WG.V)
    if m == 0:
        e = e0
    elif m == 1:
        e = e1
    else:
        e = fen.Expression('((2*m-1)*x[0]*P - (m - 1)*Q) / m', degree=2, m=m, P=E[m-1], Q=E[m-2])
    ei = fen.interpolate(e, WG.V)
    E.append(ei)

In [5]:
M = WG.get_M(tosparse=False) # Only valid if WG.eps = 1.0!
def inner(v, w):
    #return (v.vector()*(M*w.vector())).sum()
    return (v.vector()*(w.vector())).sum()
def EEv(E, v, k):
    inner_Ev = [inner(e, v) for e in E[:k-1]]
    f = fen.Function(WG.V)
    f.vector()[:] = 0
    for i in range(k-1):
        f.vector()[:] = f.vector()[:] + E[i].vector()[:] * inner_Ev[i]
    return f
def vvA(v, A, k, N):
    inner_vA = [inner(v, a) for a in A[k:N]]
    F = []
    for j in range(k, N):
        f = fen.Function(WG.V)
        f.vector()[:] = 0
        for i in range(len(inner_vA)):
            f.vector()[:] = f.vector()[:] + v.vector()[:] * inner_vA[i]
        F.append(f)
    return F

In [6]:
R = np.zeros((N, N))
A_ = []
A_[:] = A[:]
for k in range(N):
    e = E[k]
    a = A_[k]
    rho = pow(inner(a, a), 0.5)
    R[k, k] = rho
    alpha = inner(e, a)
    s = 1 if alpha == 0 else - alpha / abs(alpha)
    e.vector()[:] = s * e.vector()[:]
    E[k] = e
    v = fen.Function(WG.V)
    v.vector()[:] = rho * e.vector()[:] - a.vector()[:]
    v.vector()[:] = v.vector()[:] - EEv(E, v, k).vector()[:]
    sigma = pow(inner(a, a), 0.5)
    if sigma == 0:
        v.vector()[:] = e.vector()[:]
    else:
        v.vector()[:] = v.vector()[:] / sigma
    for j in range(k+1, N):
        A_[j].vector()[:] = A_[j].vector()[:] - 2*vvA(v, A_, k, N)[j-k].vector()[:]
    for j in range(k+1, N):
        R[k, j] = inner(e, A_[j])
    for j in range(k, N):
        A_[j].vector()[:] = A_[j].vector()[:] - e.vector()[:] * R[k, j]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in multiply


In [7]:
R.T @ R

array([[  5.21687631e+000,   1.14572274e+006,  -3.83255714e+010,
          1.28202869e+015,  -4.28851417e+019,               nan],
       [  1.14572274e+006,   4.20812553e+015,   4.77987632e+041,
         -1.34025410e+047,   3.75800741e+052,               nan],
       [ -3.83255714e+010,   4.77987632e+041,   7.57702494e+072,
         -3.03753273e+107,   4.10753931e+122,               nan],
       [  1.28202869e+015,  -1.34025410e+047,  -3.03753273e+107,
          5.49985451e+148,   1.04608445e+202,               nan],
       [ -4.28851417e+019,   3.75800741e+052,   4.10753931e+122,
          1.04608445e+202,   4.36482853e+263,               nan],
       [              nan,               nan,               nan,
                      nan,               nan,               inf]])

In [10]:
inner(A[1], A[1])

5.899358075034189e+20